# ECFP Feature Engineering for Traditional ML

Our goal in this step is to convert the list of SMILES strings from `clintox.csv` into a numerical format that traditional machine learning models, like  Logistic Regression or Random Forest

The most common and effective way to do this is by using molecular fingerprints. We will use Extended-Connectivity Fingerprints (ECFP), which are called Morgan Fingerprints in the RDKit library.

What is a Morgan Fingerprint (ECFP)?

If we think of a molecule as a graph of atoms (nodes) and bonds (edges).

    A fingerprint algorithm "looks" at every atom in the molecule.

    It identifies all the small sub-structures surrounding that atom up to a certain distance (this distance is the radius).

    It hashes each unique sub-structure into an ID.

    It creates a long vector (e.g., of 2048 bits) of all zeros.

    If a specific sub-structure is present anywhere in the molecule, it "flips" the bit at that sub-structure's ID from 0 to 1.

The result is a binary vector (a list of 0s and 1s) of a fixed length (e.g., 2048) for every molecule. This vector is our feature set (X).

    radius: We will use a radius of 2. This is often called "ECFP4" (diameter 4). It's a standard choice that captures enough local information without being too specific.

    nBits: We will use nBits=2048. This is the length of our vector. It's large enough to avoid most "hash collisions" (where two different sub-structures get the same ID) for a dataset of this size.


### So, let's code it

#### 1- Importing Libraries

In [27]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors
from rdkit.DataStructs import cDataStructs # For explicit bit vector conversion
import warnings

# Suppress RDKit warnings
warnings.filterwarnings('ignore')
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


#### Load the dataset and set values of the `radius` and the `bits`

In [28]:
# Radius for the Morgan Fingerprint (radius=2 is standard for ECFP4)
FP_RADIUS = 2

# Number of bits in the fingerprint vector
FP_NBITS = 2048

try:
    data = pd.read_csv("C:/Users/jmurd/OneDrive/Escritorio/UPV/2025-2026 (NTNU)/Data Science/Proyecto/ClinTox/data/clintox.csv")
    print(f"Successfully loaded clintox.csv. Found {len(data)} compounds.")
except FileNotFoundError:
    print("Error: clintox.csv not found. Make sure it's in the same directory.")
    


Successfully loaded clintox.csv. Found 1477 compounds.


#### 3- Define our function to transform the smiles strings into Morgan Fingerprint (ECFP)

In [30]:
def generate_ecfp(smiles_string, radius, nbits):
        mol = Chem.MolFromSmiles(smiles_string)
        if mol is None:
            # print(f"Warning: Could not parse SMILES: {smiles_string}")
            return None
        
        fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=nbits)
        
        fp_array = np.zeros((nbits,), dtype=np.int8)
        cDataStructs.ConvertToNumpyArray(fp, fp_array)
        return fp_array

#### 4-Process the entire dataset

In [31]:
fingerprints_list = []
labels_list = []
    
# Suppress warnings for invalid SMILES during processing loop
invalid_smiles_count = 0

for index, row in data.iterrows():
    smiles = row['SMILES']
    label = row['CT_TOX']
    fp_array = generate_ecfp(smiles, FP_RADIUS, FP_NBITS)
        
    if fp_array is not None:
        fingerprints_list.append(fp_array)
        labels_list.append(label)
    else:
        invalid_smiles_count += 1

if invalid_smiles_count > 0:
    print(f"Warning: Skipped {invalid_smiles_count} invalid/unparsable SMILES strings.")

# ---Create Final X and y Matrices ---
X = np.array(fingerprints_list)
y = np.array(labels_list)
    
print("...Fingerprint generation complete.")
print(f"Shape of X (features matrix): {X.shape}")
print(f"Shape of y (labels vector): {y.shape}")

# --- Save the results in .npz format ---
output_filename = 'clintox_ecfp_features.npz'
np.savez_compressed(output_filename, X=X, y=y)
    
print(f"\nSuccessfully saved features (X) and labels (y) to '{output_filename}'")

...Fingerprint generation complete.
Shape of X (features matrix): (1477, 2048)
Shape of y (labels vector): (1477,)

Successfully saved features (X) and labels (y) to 'clintox_ecfp_features.npz'


-------------------------------------------

### We have our desired X and Y features, so we can proced with the train and test split to train basic machine learning models

To divide the data we will use the `Scaffold` split
- `Why?`  A standard "random split" is not good for molecular data. Molecules often come in families that share the same core "scaffold." A random split might put "Molecule A" in the training set and its nearly identical "Cousin B" in the test set. The model would easily get "Cousin B" right, giving you a falsely high score.
- A scaffold split analyzes the core structure of every molecule. It guarantees that all molecules sharing the same scaffold end up in the same set (e.g., all in training, or all in testing). This is a much harder, more realistic test of the model's ability to generalize to new, unseen chemical families

In [36]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from collections import defaultdict
from sklearn.model_selection import train_test_split
import warnings

try:
    data_npz = np.load('clintox_ecfp_features.npz')
    X = data_npz['X']
    y = data_npz['y']
except FileNotFoundError:
    print("Error: 'clintox_ecfp_features.npz' not found")
    raise

# Load the original CSV to get the SMILES
print("Loading SMILES from clintox.csv...")
data_csv = pd.read_csv("C:/Users/jmurd/OneDrive/Escritorio/UPV/2025-2026 (NTNU)/Data Science/Proyecto/ClinTox/data/clintox.csv") 

# --- 2. Synchronize SMILES with X and y ---
# (Repeat the same filter to ensure X, y, and valid_smiles match)
valid_smiles = []
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for index, row in data_csv.iterrows():
        mol = Chem.MolFromSmiles(row['SMILES'])
        if mol is not None:
            valid_smiles.append(row['SMILES'])

assert len(valid_smiles) == len(X), "Synchronization error!"
print(f"Data synchronized. {len(valid_smiles)} valid molecules ready.")





Loading SMILES from clintox.csv...
Data synchronized. 1477 valid molecules ready.


We split our data into three `sets—Train (80%)`, `Validation (10%)`, and `Test (10%)` to properly evaluate our model's performance. The Train Set is the "study material" the model uses to learn patterns. The Validation Set acts as a "practice exam," which we use to tune the model's settings (hyperparameters) and make decisions (like which model is best) without "cheating." Finally, the Test Set is the "final exam" that we only use once at the very end. This set provides the final, honest AUROC score and proves our model can generalize to new, unseen data, as outlined in our proposal .

In [37]:
# --- Manual Scaffold Split Implementation ---
def get_scaffold(smiles):
    """Obtains the Murcko scaffold of a SMILES."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        scaffold = MurckoScaffold.GetScaffoldForMol(mol)
        scaffold_smiles = Chem.MolToSmiles(scaffold, isomericSmiles=False)
        # If there is no scaffold (e.g., molecule 'C'), use the original SMILES
        return scaffold_smiles if scaffold_smiles else smiles
    except:
        # Fallback for RDKit versions without GetScaffoldForMol
        return smiles

# Group indices by scaffold
scaffold_groups = defaultdict(list)
for i, smiles in enumerate(valid_smiles):
    scaffold = get_scaffold(smiles)
    scaffold_groups[scaffold].append(i) # Store the index (i)
# Now we have a dictionary: {'scaffold_smiles': [list_of_indices]}

In [39]:

# Split the *groups* of scaffolds, not the molecules
# We want 80% train, 10% valid, 10% test 
scaffold_list = list(scaffold_groups.keys())
train_scaffolds, test_val_scaffolds = train_test_split(scaffold_list, test_size=0.2, random_state=42)
valid_scaffolds, test_scaffolds = train_test_split(test_val_scaffolds, test_size=0.5, random_state=42)

# Function to reconstruct datasets from indices
def get_data_from_scaffolds(scaffold_keys):
    indices = [idx for key in scaffold_keys for idx in scaffold_groups[key]]
    return X[indices], y[indices]

# Reconstruct datasets
X_train, y_train = get_data_from_scaffolds(train_scaffolds)
X_valid, y_valid = get_data_from_scaffolds(valid_scaffolds)
X_test, y_test = get_data_from_scaffolds(test_scaffolds)

# --- 4. Split Results ---
print("\n---Split Completed! ---")
print(f"X_train shape: {X_train.shape} | y_train shape: {y_train.shape}")
print(f"X_valid shape: {X_valid.shape} | y_valid shape: {y_valid.shape}")
print(f"X_test shape:  {X_test.shape}  | y_test shape:  {y_test.shape}")
print(f"Total molecules: {len(y_train) + len(y_valid) + len(y_test)} (should be {len(X)})")

# --- 5. Save the Split Data ---
output_split_filename = 'clintox_ecfp_split.npz'
np.savez_compressed(
    output_split_filename,
    X_train=X_train, y_train=y_train,
    X_valid=X_valid, y_valid=y_valid,
    X_test=X_test, y_test=y_test
)
print(f"\nData saved to '{output_split_filename}'")


---Split Completed! ---
X_train shape: (1175, 2048) | y_train shape: (1175,)
X_valid shape: (146, 2048) | y_valid shape: (146,)
X_test shape:  (156, 2048)  | y_test shape:  (156,)
Total molecules: 1477 (should be 1477)

Data saved to 'clintox_ecfp_split.npz'


--------------------------------------------

Now we have our data divided into `train`, `test` and `valid`, so we can proceed with the creation of the models

### The first model will be a `Random Forest`
- Why?  A Random Forest is a non-linear model. It can learn complex, conditional relationships, like "this substructure is toxic, but only if this other substructure is also present." This is much more like real-world biology.
- Robustness: RF is an "ensemble" model. It builds hundreds of simple "decision trees" and lets them vote on the answer. This makes it very robust to noise and less prone to overfitting than a single, deep decision tree.

In [40]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix
import time

In [ ]:
print("Loading pre-split data from 'clintox_ecfp_split.npz'...")
try:
    data = np.load('clintox_ecfp_split.npz')
    X_train, y_train = data['X_train'], data['y_train']
    X_valid, y_valid = data['X_valid'], data['y_valid']
    X_test, y_test = data['X_test'], data['y_test']
    
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'clintox_ecfp_split.npz' not found. Run Step 2 first.")
    raise

# --- Define the Model and GridSearch Parameters ---

# Initialize the RF Classifier
# We MUST include class_weight='balanced' to handle imbalance 
rf = RandomForestClassifier(class_weight='balanced', random_state=42)

# Define the "grid" of hyperparameters to test
# This is a small grid to run quickly. You can expand it.
param_grid = {
    'n_estimators': [100, 300, 500, 1000],       
    'max_depth': [10, 20, 50, 100, None],          
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 5, 10],
    'max_features': ['sqrt', 0.25, 0.5] # 'sqrt' is default, 0.5 means 50% of features
}

# Initialize GridSearch
# We tell it to use 'roc_auc' as its scoring metric 
# n_jobs=-1 uses all available CPU cores to speed up the search
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,  # 3-fold cross-validation on the training data
    n_jobs=-1,
    verbose=1 # Shows progress
)

# --- 3. Run the Hyperparameter Search ---
print("\nStarting GridSearch for Random Forest...")
start_time = time.time()

# Fit the grid search on the TRAINING data
# GridSearchCV will find the best params using cross-validation
grid_search.fit(X_train, y_train)

end_time = time.time()
print(f"...GridSearch complete in {end_time - start_time:.2f} seconds.")

# --- 4. Show Best Parameters and Evaluate on Validation Set ---

# This is the best model found by the search
best_rf = grid_search.best_estimator_

print(f"\nBest parameters found: {grid_search.best_params_}")

# Evaluate this best model on the VALIDATION set
probs_valid_rf = best_rf.predict_proba(X_valid)[:, 1]
auc_valid_rf = roc_auc_score(y_valid, probs_valid_rf)
print(f"Best Model Validation AUROC: {auc_valid_rf:.4f}")

# --- 5. Final Evaluation on the TEST Set ---

# Now we get the *final*, unbiased score from the test set
probs_test_rf = best_rf.predict_proba(X_test)[:, 1]
auc_test_rf = roc_auc_score(y_test, probs_test_rf)

print(f"\n--- Final Random Forest Performance ---")
print(f"Test AUROC: {auc_test_rf:.4f}")


Loading pre-split data from 'clintox_ecfp_split.npz'...
Data loaded successfully.

Starting GridSearch for Random Forest...
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
